# Projet

On utlisera l'API de Github

Requetes utiles :
- Recuperer les répertoires en top tendences sur github
- Recuperer les X répertoires en tendences dans toutes les languages
- Recuperer les X répertoires les plus "starred" pour chaque languages
- Recuperer les X répertoires les plus "starred" pour une collection
- Recuperer les collections contenant un mot
- Recuperer les répertoires d'une personne
- Recuperer les répertoires appartenant à une personne (pas fork)
- Recuperer les répertoires forkés par une personne
- Recuperer les personnes ayant forkés un répertoires
- Recuperer les personnes ayant "starred" un repertoire

- on ne gere pas les watchers, c'est un peu inutile comme fonctionnalité

In [6]:
!pip install 'pymongo[srv]'

In [8]:
from pymongo import MongoClient

client = MongoClient("mongo")
database = client['projet']

database.command("serverStatus")
print(serverStatusResult)

{'host': 'fdf2efd7045d', 'version': '4.4.3', 'process': 'mongod', 'pid': 1, 'uptime': 26406.0, 'uptimeMillis': 26406234, 'uptimeEstimate': 26406, 'localTime': datetime.datetime(2021, 2, 19, 13, 34, 27, 495000), 'asserts': {'regular': 0, 'warning': 0, 'msg': 0, 'user': 4, 'rollovers': 0}, 'connections': {'current': 2, 'available': 838858, 'totalCreated': 14, 'active': 1, 'exhaustIsMaster': 0, 'exhaustHello': 0, 'awaitingTopologyChanges': 0}, 'electionMetrics': {'stepUpCmd': {'called': 0, 'successful': 0}, 'priorityTakeover': {'called': 0, 'successful': 0}, 'catchUpTakeover': {'called': 0, 'successful': 0}, 'electionTimeout': {'called': 0, 'successful': 0}, 'freezeTimeout': {'called': 0, 'successful': 0}, 'numStepDownsCausedByHigherTerm': 0, 'numCatchUps': 0, 'numCatchUpsSucceeded': 0, 'numCatchUpsAlreadyCaughtUp': 0, 'numCatchUpsSkipped': 0, 'numCatchUpsTimedOut': 0, 'numCatchUpsFailedWithError': 0, 'numCatchUpsFailedWithNewTerm': 0, 'numCatchUpsFailedWithReplSetAbortPrimaryCatchUpCmd':

###  Utilisateur

* Infos d'un user
* Repos d'un user
* Gist d'un user
* Stars du user
* Repos les plus populaires (les + vus) : voir si possible
* Repos les plus forks
* Repos le plus starred
* Forks d'un user
* Répos owns du user

### Topics

* Topics existant
* Topics contenant un mot
* Tags associés à un topic => Pour chaque repo, on recup les tags et on compte le nombre qu'un tag apparaît (soit aggregate du début à la fin normalement)
* Repos d'un topic
* Languages le plus utilisés dans un topic : voir si c'est possible